In [8]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 

from scipy.stats import zscore
from scipy.stats import randint

import lightgbm as lgb
import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

import joblib

%matplotlib inline

In [9]:
df = pd.read_csv('first inten project.csv')

In [10]:
def get_outlier_indices(df, columns=None, threshold=3):
    outlier_indices = set() 

    if columns is None:
        columns = df.select_dtypes(include=['number']).columns

    for col in columns:
        z_scores = zscore(df[col])
        col_outliers = df[(z_scores > threshold) | (z_scores < -threshold)].index
        
        outlier_indices.update(col_outliers)

    return list(outlier_indices)


columns_to_check = ['number of adults', 'number of children', 'average price ', 'number of weekend nights',
                    'number of week nights','special requests', 'P-C', 'P-not-C', 'lead time'] 


outlier_indices = get_outlier_indices(df, columns=columns_to_check)
len(outlier_indices)
#I tried with threshold 4, but the accuracy was better with 3

2933

In [11]:
df_cleaned = df.drop(outlier_indices)

In [16]:
columns = df_x_encoded.columns
print(columns)

Index(['number of adults', 'number of children', 'number of weekend nights',
       'number of week nights', 'car parking space', 'lead time', 'repeated',
       'P-C', 'P-not-C', 'average price ', 'special requests',
       'type of meal_Meal Plan 1', 'type of meal_Meal Plan 2',
       'type of meal_Meal Plan 3', 'type of meal_Not Selected',
       'room type_Room_Type 1', 'room type_Room_Type 2',
       'room type_Room_Type 3', 'room type_Room_Type 4',
       'room type_Room_Type 5', 'room type_Room_Type 6',
       'room type_Room_Type 7', 'market segment type_Aviation',
       'market segment type_Complementary', 'market segment type_Corporate',
       'market segment type_Offline', 'market segment type_Online'],
      dtype='object')


In [12]:
df_x = df_cleaned.drop(columns=['booking status', 'Booking_ID', 'date of reservation'])
df_x_encoded = pd.get_dummies(df_x, columns=['type of meal', 'room type', 'market segment type'])
df_y = pd.get_dummies(df_cleaned, columns=['booking status'])
df_x = df_cleaned.drop(columns=['booking status', 'Booking_ID'])
X = df_x_encoded 
y = df_cleaned['booking status']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(X_train, y_train)
y_pred_forest = random_forest.predict(X_test)

In [7]:
print("Random Forest Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_forest)}")
print("Classification Report:")
print(classification_report(y_test, y_pred_forest))

Random Forest Performance:
Accuracy: 0.8874231749362914
Classification Report:
              precision    recall  f1-score   support

    Canceled       0.84      0.79      0.81      2090
Not_Canceled       0.91      0.93      0.92      4581

    accuracy                           0.89      6671
   macro avg       0.87      0.86      0.87      6671
weighted avg       0.89      0.89      0.89      6671



In [9]:
joblib.dump(random_forest, 'random_forest_model.pkl')

['random_forest_model.pkl']